In [1]:
#importing libararies
import pandas as pd
import numpy as np
import os
from __future__ import print_function
from mailmerge import MailMerge
from datetime import date
import docx
import import_ipynb
import glob
import tkinter
from tkinter import filedialog
import warnings
warnings.filterwarnings("ignore")


def format_sheet(file_path:str,template_path:str,sheet_num_template=0,run=0):
    """
    file_path: the file you want to design, must bx xlsx
    template_path: the teplate of the design, must bx xlsx
        """
    # format sheet
    xlsx=pd.ExcelWriter(file_path, engine='openpyxl', mode='a')
    for i in range(len(xlsx.book.worksheets)):
        new_sheet=xlsx.book.worksheets[run]
        template=pd.ExcelWriter(template_path,engine='openpyxl', mode='a')
        default_sheet=template.book.worksheets[sheet_num_template]
        from copy import copy
        new_sheet.sheet_view.rightToLeft=True
        for row in default_sheet.rows:
            for cell in row:
                new_cell=new_sheet.cell(row=cell.row, column=cell.col_idx)
                if cell.has_style and new_cell.value != None:
                    new_cell.font=copy(cell.font)
                    new_cell.border=copy(cell.border)
                    new_cell.fill=copy(cell.fill)
                    new_cell.number_format=copy(cell.number_format)
                    new_cell.protection=copy(cell.protection)
                    new_cell.alignment=copy(cell.alignment)
        from openpyxl.utils import get_column_letter
        for i in range(default_sheet.max_column):
            new_sheet.column_dimensions[get_column_letter(i + 1)].width=default_sheet.column_dimensions[
                get_column_letter(i + 1)].width
    
    xlsx.save()
def change_names_and_order(file_path:str,df:pd.DataFrame):
    """
    name of columns in names must be "old_name", "new_name"
        """
    match_names=pd.read_excel(file_path)
    match_names.index=match_names.old_name
    dict_names=match_names.to_dict()['new_name']
    ls_names=match_names['new_name'].tolist()
    df.rename(columns=dict_names, inplace=True)
    df=df[ls_names]
    return df

In [88]:

tkinter.Tk().withdraw() # prevents an empty tkinter window from appearing

transction_folder_path = filedialog.askdirectory()
permanant_folder_path = filedialog.askdirectory()

In [89]:
#reading the check file
filepath_check = transction_folder_path+"\*MCI*"
txt_check = glob.glob(filepath_check)
for textfile in txt_check:
    mci = pd.read_excel(textfile ,skiprows=1 ,usecols="A:J")
    


#reading the risk country file
filepath_check = permanant_folder_path+"\*מדינות*"
txt_check = glob.glob(filepath_check)
for textfile in txt_check:
    rc = pd.read_excel(textfile)

#reading the rates file
filepath_check = permanant_folder_path+"\*שערים*"
txt_check = glob.glob(filepath_check)
for textfile in txt_check:
    rates = pd.read_excel(textfile ,sheet_name="CUR CONV",usecols="F:G",skiprows=1)
    

#reading the report number generator file
filepath_check = transction_folder_path+"\*מחולל*"
txt_check = glob.glob(filepath_check)
for textfile in txt_check:
    report_num = pd.read_excel(textfile)
    
#reading the report content file
filepath_check = transction_folder_path+"\*content*"
txt_check = glob.glob(filepath_check)
for textfile in txt_check:
    content = pd.read_excel(textfile)  
    
#reading the report content file
filepath_check = transction_folder_path+"\*מעקב*"
txt_check = glob.glob(filepath_check)
for textfile in txt_check:
    reported = pd.read_excel(textfile)   

In [90]:
mci = mci.rename(columns = {'Transaction':'deal number','Amount':'amount','סכום בש"ח':'currency','שם השולח':'nis sum','ID Num':'clients name','שם הנמען':'id number','יעד':'owner name','Unnamed: 9':'dest country'})

In [91]:
#rounding the amount and nis sum:
mci = mci.dropna(subset = 'amount')
mci['nis sum'] = mci['nis sum'].astype(int)
mci['amount'] = mci['amount'].astype(int)
rc['English short name (using title case)'] = rc['English short name (using title case)'].str.lower()
mci['dest country'] = mci['dest country'].str.lower()
# find risk transactions gmt
mci['dest country']=mci['dest country'].str.strip()
risk_type1_mci=mci.merge(rc[['English short name (using title case)','סיכון/ללא דיווח רגיל']],left_on="dest country",right_on="English short name (using title case)")
risk_type1_mci = risk_type1_mci[risk_type1_mci['סיכון/ללא דיווח רגיל'].notna()]
mci['citizenship'] = mci['id number'].str.split(' ', 1, expand=True)[0]
risk_type2_mci = mci.merge(rc[['Alpha-2 code','סיכון/ללא דיווח רגיל']], left_on = 'citizenship',right_on = 'Alpha-2 code', how = 'left')
risk_type2_mci = risk_type2_mci[risk_type2_mci['סיכון/ללא דיווח רגיל'].notna()]
all_risk=pd.concat([risk_type1_mci,risk_type2_mci]).drop_duplicates("deal number")

In [92]:
#building three kinds of monitoring data frames
monitoring_name = mci.groupby('clients name').agg({'nis sum':'sum','deal number':'nunique'})
monitoring_ow = mci.groupby('owner name').agg({'nis sum':'sum','clients name':'nunique'})

#filtering need to report transctions:
monitoring_name_2report = monitoring_name[((monitoring_name['nis sum'] >= 50000) & (monitoring_name['deal number'] > 1)) | ((monitoring_name['nis sum'] >= 47000) & (monitoring_name['nis sum'] < 50000))]

#adding the report type:
monitoring_name_2report['report type'] = np.where((monitoring_name_2report['nis sum'] >= 50000) , 'p', 'd')



In [93]:
#building three kinds of monitoring data frames
monitoring_name_risk = all_risk.groupby('clients name').agg({'nis sum':'sum','deal number':'nunique'})
monitoring_ow_risk = all_risk.groupby('owner name').agg({'nis sum':'sum','clients name':'nunique'})

#filtering need to report transctions:
monitoring_name_risk_2report = monitoring_name_risk[((monitoring_name_risk['nis sum'] >= 5000) & (monitoring_name_risk['deal number'] > 1)) | ((monitoring_name_risk['nis sum'] >= 4700) & (monitoring_name_risk['nis sum'] < 5000))]

#adding the report type:
monitoring_name_risk_2report['report type_risk'] = np.where((monitoring_name_risk_2report['nis sum'] >= 5000) , 'pr', 'dr')


In [94]:
#concating the client to report df's
all_reports = monitoring_name_2report.append(monitoring_name_risk_2report)
all_reports = all_reports.reset_index()
all_reports = all_reports.merge(reported['שם'], left_on = 'clients name', right_on = 'שם' , how = 'left')
all_reports = all_reports[all_reports['שם'].isna()]


In [95]:
#shared destanation monitoring
monitoring_ow_2report = monitoring_ow[(monitoring_ow['nis sum'] >= 50000) & (monitoring_ow['clients name'] > 1)]
monitoring_ow_2report = monitoring_ow_2report.rename(columns = {'clients name':'clients count'})

#checking if a client is already reported in previous test
monitoring_ow_2report_clients = monitoring_ow_2report.merge(mci[['clients name','owner name']],on = 'owner name' , how = 'left')
monitoring_ow_2report_clients = monitoring_ow_2report_clients.merge(all_reports['clients name'] , on = 'clients name', how = 'left', suffixes=['','_pd'])
try:
    monitoring_ow_2report_clients = monitoring_ow_2report_clients[monitoring_ow_2report_clients['clients name_pd'].isna()]
except KeyError:
    print('no match between this p or d reported client and shared owner risk clients')
try:   
    monitoring_ow_2report_clients = monitoring_ow_2report_clients.merge(reported['שם'], left_on = 'clients name', right_on = 'שם', how = 'left')
    monitoring_ow_2report_clients = monitoring_ow_2report_clients[monitoring_ow_2report_clients['שם'].isna()]
except:
    print('no match between last 3 month reported and shared owner risk clients')

#appending the shared owner clients to the all reports DF
monitoring_ow_2report_clients['report type ow'] = 'm'
monitoring_ow_2report_clients = monitoring_ow_2report_clients.drop_duplicates('owner name')
all_reports = all_reports.append(monitoring_ow_2report_clients)


no match between this p or d reported client and shared owner risk clients


In [96]:
#shared destanation monitoring
monitoring_ow_2report_risk = monitoring_ow_risk[(monitoring_ow_risk['nis sum'] >= 5000) & (monitoring_ow_risk['clients name'] > 1)]
monitoring_ow_2report_risk = monitoring_ow_2report_risk.rename(columns = {'clients name':'clients count'})

#checking if a client is already reported in previous tests
monitoring_ow_2report_risk = monitoring_ow_2report_risk.merge(all_risk[['clients name','owner name']],on = 'owner name' , how = 'left')
monitoring_ow_2report_risk_clients = monitoring_ow_2report_risk.merge(all_reports['clients name'] , on = 'clients name', how = 'left', suffixes=('','_pd'))
try:
    monitoring_ow_2report_risk_clients = monitoring_ow_2report_risk_clients[monitoring_ow_2report_risk_clients['clients name_pd'].isna()]
except KeyError:
    print('no match between this p or d reported client and shared owner risk clients')
try:
    monitoring_ow_2report_risk_clients = monitoring_ow_2report_risk_clients.merge(reported['שם'], left_on = 'clients name', right_on = 'שם', how = 'left')
    monitoring_ow_2report_risk_clients = monitoring_ow_2report_risk_clients[monitoring_ow_2report_risk_clients['שם'].isna()]
except:
    print('no match between last 3 month reported and shared owner risk clients')

#appending the risk hared owner clients to the all reports DF
monitoring_ow_2report_risk_clients['report type ow risk'] = 'mr'
monitoring_ow_2report_risk_clients = monitoring_ow_2report_risk_clients.drop_duplicates('owner name')
all_reports = all_reports.append(monitoring_ow_2report_risk_clients)



no match between this p or d reported client and shared owner risk clients


In [97]:
#joining all the report types into one list
all_reports = all_reports.fillna('')
all_reports['final type'] = all_reports['report type'] + all_reports['report type_risk'] + all_reports['report type ow'] + all_reports['report type ow risk']

In [98]:
#summarizing the data frame for mailing
all_reports4word = all_reports[['clients name','final type','owner name']]


if all_reports4word.empty == True:
    print(' אין דיווחים בלתי רגילים!')
    exit(1)
    


 אין דיווחים בלתי רגילים!


In [99]:
#adding the country of dest details
all_reports4word = all_reports4word.merge(mci[['clients name','dest country','citizenship','id number']], on = 'clients name',how = 'left')
all_reports4word = all_reports4word.drop_duplicates('clients name')
#adding the hebrew name of the countries
all_reports4word = all_reports4word.merge(rc[['English short name (using title case)','עברית']], left_on = 'dest country', right_on = 'English short name (using title case)', how = 'left')
all_reports4word = all_reports4word.drop_duplicates('clients name')
all_reports4word = all_reports4word.rename(columns = {'עברית':'dest country heb'})

#adding the citizenship name
all_reports4word = all_reports4word.merge(rc[['Alpha-2 code','עברית']], left_on = 'citizenship', right_on = 'Alpha-2 code', how = 'left')
all_reports4word = all_reports4word.drop_duplicates('clients name')
all_reports4word = all_reports4word.rename(columns = {'עברית':'citizenship heb'})

In [100]:
#merging the report content
all_reports4word = all_reports4word.merge(content, left_on = 'final type' , right_on = 'type', how = 'left')
#all_reports4word['risk_full_content'] ='הכספים נשלחו ל'+all_reports4word['עברית'] + ' ,מדינה המוגדרת בסיכון גבוה.'
try:
    all_reports4word['risk_full_content'] =np.where((all_reports4word['final type'] == 'mr')|(all_reports4word['final type'] == 'pr')|(all_reports4word['final type'] == 'dr')|(all_reports4word['final type'] == 'mkr')|(all_reports4word['final type'] == 'kpr')|(all_reports4word['final type'] == 'kdr') ,'הכספים נשלחו ל'+all_reports4word['dest country heb'] + ' ,מדינה המוגדרת בסיכון גבוה.','')
except KeyError:
    print('אין דיווחים בסיכון')

In [101]:
#importing the report number needed
report_id = (report_num.iloc[1,5] + 1).astype(int)
all_reports4word['report_id'] = report_id + range(len(all_reports4word.index))
all_reports4word['report_name'] = report_num.iloc[3,1].rsplit('-',1)[0]

In [102]:
all_reports4word = all_reports4word.fillna('')
all_reports4word = all_reports4word.reset_index()
all_reports4word.to_excel("monitoring.xlsx")

In [103]:
filepath_check = transction_folder_path+"\*טמפלייט*"
txt_check = glob.glob(filepath_check)
for textfile in txt_check:
    template = textfile
document = MailMerge(template)
print(document.get_merge_fields())

{'id_number', 'report_id', 'citizenship_heb', 'clients_name', 'Content', 'risk_full_content', 'Title'}


In [104]:
for i in all_reports4word.index:
     #template1 = 'test1.docx'
    document = MailMerge(template)
    document.merge(
        clients_name = str(all_reports4word['clients name'][i]),
        Title = str(all_reports4word['Title'][i]),
        report_id = str(all_reports4word['report_id'][i]),
        Content = str(all_reports4word['Content'][i]),
        risk_full_content = str(all_reports4word['risk_full_content'][i]),
        citizenship_heb = str(all_reports4word['citizenship heb'][i]),
        id_number = str(all_reports4word['id number'][i])
       
        )
    

    output =  str(all_reports4word['report_name'][i]) + "-" + str(all_reports4word['report_id'][i]) + '.docx'
    document.write(output)

In [105]:
#filtering outbound reports
sent_report = all_reports4word[(all_reports4word['final type'] == 'p') |(all_reports4word['final type'] == 'd') |(all_reports4word['final type'] == 'dr') |(all_reports4word['final type'] == 'pr') | (all_reports4word['final type'] == 'ppr')]

#creating a data frames with only need to report transctions
sent_report_tr = mci[mci['clients name'].isin(sent_report['clients name'])]

#leaving only the imporatant columns
sent_report_tr = sent_report_tr[['deal number','Date','dest country','clients name','owner name','amount','currency','nis sum','id number']]

#creating an excel shhet with the ountbound reports
i=0
for_word_table=[]
xlsx=pd.ExcelWriter('monitoring.xlsx', engine='openpyxl', mode='a',if_sheet_exists='overlay')
#round the sum in ils
sent_report_tr["nis sum"]=sent_report_tr["nis sum"].round()

#creating a unique list of the sender names
ls_customer=sent_report_tr["clients name"].unique().tolist()

#loop each sender transctions in a unique table
for name in ls_customer:
    df=sent_report_tr[sent_report_tr["clients name"]==name]  
    data = [["סהכ",df["amount"].sum(),df["nis sum"].sum()]]
    last_row = pd.DataFrame(data, columns = ["clients name",'amount', "nis sum"])
    orderd=pd.concat([df,last_row])
    orderd.to_excel(xlsx, sheet_name='Reports',startrow=i,index=False)
    for_word_table.append(orderd)
    i=i+len(orderd)+2
            
    xlsx.save()

In [106]:
#indexing the outbound_report data frame
info2r = sent_report.set_index('clients name')
info2r_nona = info2r[info2r.index.notnull()]

for title in ls_customer:
    
    data = sent_report_tr[sent_report_tr['clients name'] == title].round()
    table = [["סהכ",data["nis sum"].sum().round(),data['amount'].sum()]]
    last_row = pd.DataFrame(table, columns = ["clients name", "nis sum",'amount'])
    orderd=pd.concat([data,last_row])
    orderd=orderd.fillna(' ')
    
    orderd=change_names_and_order(transction_folder_path + r"\change_names.xlsx",orderd)
    
    orderd['תאריך פעולה'] = orderd['תאריך פעולה'].astype(str)
    
    doc = docx.Document(str(info2r_nona['report_name'].loc[title]) + "-" + str(info2r_nona['report_id'].loc[title]) + ".docx")  

    doc.add_page_break()
    t = doc.add_table(orderd.shape[0]+1, orderd.shape[1])
                        
              

    for j in range(orderd.shape[-1]):
        t.cell(0,j).text = orderd.columns[j]

        # add the rest of the data frame
    for i in range(orderd.shape[0]):
        for j in range(orderd.shape[-1]):
            t.cell(i+1,j).text = str(orderd.values[i,j])

    t.style = 'Grid Table 4 Accent 5'
    
    doc.save(str(info2r_nona['report_name'].loc[title]) + "-" + str(info2r_nona['report_id'].loc[title]) + ".docx")


In [107]:
#filtering only shared owner reports
sent_report_ow = all_reports4word[(all_reports4word['final type'] == 'm') |(all_reports4word['final type'] == 'mr')]

#concatacating the two kinds of outbound shared owner transction dataframes
sent_report_tr_ow = mci[mci['owner name'].isin(sent_report_ow['owner name'])]


#leaving only the imporatant columns
sent_report_tr_ow = sent_report_tr_ow[['deal number','Date','dest country','clients name','owner name','amount','currency','nis sum','id number']]

#creating a new list
j=0
for_word_table_ow=[]

#wrting an excel file containing the report transctions for each customer
xlsx=pd.ExcelWriter('monitoring.xlsx', engine='openpyxl', mode='a',if_sheet_exists='overlay')

#round the sum in ils
sent_report_tr_ow["nis sum"]=sent_report_tr_ow["nis sum"].round()

#creating a unique list of the sender names
ls_customer_ow=sent_report_tr_ow["owner name"].unique().tolist()

#loop each sender transctions in a unique table
for name in ls_customer_ow:
    df=sent_report_tr_ow[sent_report_tr_ow["owner name"]==name]  
    data = [["סהכ",df["nis sum"].sum(),df["amount"].sum()]]
    last_row = pd.DataFrame(data, columns = ["clients name", "nis sum",'amount'])
    orderd=pd.concat([df,last_row])
    orderd.to_excel(xlsx, sheet_name='Reports_ow',startrow=j,index=False)
    for_word_table_ow.append(orderd)
    j=j+len(orderd)+2
            
    xlsx.save()

In [108]:
#creating a list containing the client name and owner name
info2r_ow = sent_report_ow[['report_name','report_id','clients name']].merge(sent_report_tr_ow[['clients name','owner name']], on = 'clients name', how = 'left')
info2r_ow = info2r_ow.drop_duplicates('owner name')
info2r_ow = info2r_ow.set_index('owner name')


for title in ls_customer_ow:    
    data = sent_report_tr_ow[sent_report_tr_ow['owner name'] == title]
    table = [["סהכ",data["nis sum"].sum().round(),data['amount'].sum()]]
    last_row = pd.DataFrame(table, columns = ["owner name", "nis sum",'amount'])
    orderd=pd.concat([data,last_row])
    orderd=orderd.fillna(" ")
    orderd=change_names_and_order(transction_folder_path + r"\change_names.xlsx",orderd)
    
    doc = docx.Document(str(info2r_ow['report_name'].loc[title]) + "-" + str(info2r_ow['report_id'].loc[title]) + ".docx")  

    doc.add_page_break()
    t = doc.add_table(orderd.shape[0]+1, orderd.shape[1])                   
              

    for j in range(orderd.shape[-1]):
        t.cell(0,j).text = orderd.columns[j]

        # add the rest of the data frame
    for i in range(orderd.shape[0]):
        for j in range(orderd.shape[-1]):
            t.cell(i+1,j).text = str(orderd.values[i,j])

    t.style = 'Grid Table 4 Accent 5'
   
    doc.save(str(info2r_ow['report_name'].loc[title]) + "-" + str(info2r_ow['report_id'].loc[title]) + ".docx")
    

In [109]:
#creating an excel filies for monitoring the results
monitoring_name_excel = monitoring_name.merge(all_reports4word[['clients name','final type']], on = 'clients name', how = 'left')
monitoring_name_excel = monitoring_name_excel.merge(reported['שם'],left_on = 'clients name', right_on = 'שם', how = 'left')
monitoring_name_excel['status'] = ''
monitoring_name_excel['שם'] = monitoring_name_excel['שם'].astype(str)
for i in monitoring_name_excel.index:
    if monitoring_name_excel['שם'][i] == monitoring_name_excel['clients name'][i]:
        monitoring_name_excel['status'][i] = 'דווח בשלושת החודשים האחרונים'
        
    elif monitoring_name_excel['nis sum'][i] <=47000 :
        monitoring_name_excel['status'][i] = 'סך עסקאות נמוך מרף הדיווח'

    elif monitoring_name_excel['deal number'][i] <= 1 :
        monitoring_name_excel['status'][i] = 'הועבר דיווח רגיל'
    else :
        monitoring_name_excel['status'][i] = monitoring_name_excel['final type'][i]
        
monitoring_name_excel = monitoring_name_excel.sort_values('nis sum', ascending = False)
monitoring_name_excel['final type'] = monitoring_name_excel['final type'].fillna(monitoring_name_excel['status'])
monitoring_name_excel = monitoring_name_excel.rename(columns= {'final type':'final status'})
monitoring_name_excel['final status'] = monitoring_name_excel['final status'].replace({'pr':'פיצול בסיכון','p':'פיצול','dr':'דירדוס בסיכון','d':'דירדוס','m':'מושך משותף','mr':'מושך משותף בסיכון'})

#xlsx = pd.ExcelWriter('monitoring.xlsx', engine='openpyxl', mode='a', if_sheet_exists='overlay')
monitoring_name_excel[['clients name','nis sum','deal number','final status']].to_excel('ניטור.xlsx', sheet_name = 'ניטור', index = False)

